In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.integrate import odeint

# Define parameters

In [2]:
Tc1 = 30 #Temperature in Celsius
Tc2 = 60 #Temperature in Celsius

#mol fraction of incoming gas
y_co = 0.4 #[mol/mol]
y_co2 = 0.2 #[mol/mol]
y_h2 = 0.38 #[mol/mol]
y_o2 = 0.01 #[mol/mol]
y_n2 = 0.01 #[mol/mol]

#Universal constants:
R = 8.314 # [m^3*Pa/K/mol]

def Kelvin_Celsius(t):
    """
    This function converts Temperature values from Kelvin to degrees Celsius
    """
    return t-273.15

def Celsius_Kelvin(t):
    """
    This function converts Temperature values from degrees Celsius to Kelvin
    """
    return t+273.15

T1 = Celsius_Kelvin(Tc1)
T2 = Celsius_Kelvin(Tc2)

Biological Parameters

In [3]:
a_G = 1000 #kJ/CmolX
print("a_G =", a_G, "[kJ/CmolX]")

a_G = 1000 [kJ/CmolX]


In [4]:
dfGR = pd.read_csv("../Data/EnthalpyGibbs.csv", encoding= 'unicode_escape', header=0, index_col="Compound", dtype = {"dfGR": float, "dfGR": float})["dfGR"]
dfHR = pd.read_csv("../Data/EnthalpyGibbs.csv", encoding= 'unicode_escape', header=0, index_col="Compound", dtype = {"dfGR": float, "dfGR": float})["dfHR"]

#Hess's law:
DrGR_carb = -4*dfGR["CO"] - 2*dfGR["Water"] + dfGR["Acetate"] + 2*dfGR["CO2"]
DrGR_hom = -4*dfGR["H2"] - 2*dfGR["CO2"] + dfGR["Acetate"] + 2*dfGR["Water"]

DrHR_carb = -4*dfHR["CO"] - 2*dfHR["Water"] + dfHR["Acetate"] + 2*dfHR["CO2"]
DrHR_hom = -4*dfHR["H2"] - 2*dfHR["CO2"] + dfHR["Acetate"] + 2*dfHR["Water"]

#Gibbs–Helmholtz equation for temperature correction
def DrGT_carb(T):
    return DrGR_carb * (T/298.15) + DrHR_carb * (1-T/298.15)
def DrGT_hom(T):
    return DrGR_hom * (T/298.15) + DrHR_hom * (1-T/298.15)

S = pd.DataFrame({
    'HomCata':  [0,    -4,  0,    -2,  2,    0,1],
    'CarbCata': [-4,   0,   0,    2,   -2,   0,1],
    'HomAna':   [0,   -2.1, -0.2, -1,  1.5,  1,0],
    'CarbAna':  [-2.1, 0,   -0.2, 1.1, -0.6, 1,0]},
    index=pd.Index(["CO","H2","NH3","CO2","H2O","X","P"]))
S

r_carb = S.CarbAna - S.CarbCata/S.CarbCata["CO"] * m_GAM/(DrGT_carb(T)/S.CarbCata["CO"])
print(f'Overall Carboxydotrophic stoichiometry at {T}K: \n {round(r_carb["CO"],2)} CO {round(r_carb["H2O"],2)} H20 {round(r_carb["NH3"],2)} NH3 + {round(r_carb["X"],2)} biomass + {round(r_carb["P"],2)} CH3COOH + {round(r_carb["CO2"],2)} CO2 = 0 \n')


r_hom  = S.HomAna - S.HomCata/S.HomCata["H2"] * m_GAM/(DrGT_hom(T)/S.HomCata["H2"])
print(f'Overall Homoacetogenic stoichiometry at {T}K: \n {round(r_hom["CO2"],2)} CO2 {round(r_hom["H2"],2)} H2 {round(r_hom["NH3"],2)} NH3 + {round(r_hom["X"],2)} biomass + {round(r_hom["P"],2)} CH3COOH + {round(r_hom["H2O"],2)} H2O = 0 \n')


\[REF] Heijnen J.J. (2010) Impact of Thermodynamic Principles in Systems Biology. In: Wittmann C., Krull R. (eds) Biosystems Engineering II. Advances in Biochemical Engineering / Biotechnology, vol 121. Springer, Berlin, Heidelberg. https://doi.org/10.1007/10_2009_63

5.1.1 Gibbs Energy for Maintenance

### Calculate Gibbs free energy and Enthalpy

<p>Given the reaction stoichiometries:</p>
<p>$ -2 CO_2 - 4 H_2 + CH_3COOH + 2 H_2O $</p>
<p>$ -4 CO - 2 H_2O + CH_3COOH + 2 CO_2 $</p>

DfG0_actn = -159.7 
DfH0_actn = -248.4
DfG0_actt = -389.9
DfH0_actt = -505.7
DfG0_co2 = -394.4
DfH0_co2 = -393.5
DfG0_co = -137.2
DfH0_co = -110.5
DfG0_h2o = -237.1
DfH0_h2o = -285.8
DfG0_h2 = 0
DfH0_h2 = 0

#Hess's law:
DrG0_carb = -4*DfG0_co - 2*DfG0_h2o + DfG0_actt + 2*DfG0_co2
DrG0_hom = -4*DfG0_h2 - 2*DfG0_co2 + DfG0_actt + 2*DfG0_h2o 

print("Gibbs energy:\n",
      " DrG0_carb =",round(DrG0_carb,1),"[kJ/mol]\n",
      " DrG0_hom =",round(DrG0_hom,1),"[kJ/mol]\n")

DrH0_carb = -4*DfH0_co - 2*DfH0_h2o + DfH0_actt + 2*DfH0_co2
DrH0_hom = -4*DfH0_h2 - 2*DfH0_co2 + DfH0_actt + 2*DfH0_h2o

print("Enthalpy:\n",
      " DrH0_carb =", round(DrH0_carb,1),"[kJ/mol]\n",
      " DrH0_hom =", round(DrH0_hom,1),"[kJ/mol]\n")

#Gibbs–Helmholtz equation for temperature correction
DrGT_carb = DrG0_carb * (T/298.15) + DrH0_carb * (1-T/298.15)
DrGT_hom = DrG0_hom * (T/298.15) + DrH0_hom * (1-T/298.15)

print("Temperature corrected Gibbs energy:\n",
      " DrGT_carb =",round(DrGT_carb,1),"[kJ/mol]\n",
      " DrGT_hom =",round(DrGT_hom,1),"[kJ/mol]")

#dropped concentration correction, in accordance with book

Reaction-specific substrate requirement for growth:

In [5]:
a_CO = a_G/(DrGT_carb(T1)/4) #based on stoichiometry of electron donor
a_H2 = a_G/(DrGT_hom(T1)/4) #based on stoichiometry of electron donor

print("Substrate required for growth:\n",
      "a_CO = ",round(a_CO,1), "[molCO/CmolX]\n",
      "a_H2 = ",round(a_H2,1), "[molH2/CmolX]")

Substrate required for growth:
 a_CO =  -17.3 [molCO/CmolX]
 a_H2 =  -26.5 [molH2/CmolX]


## Determine anabolic reaction, biomass formation stoichiometry

<p>With an apporximated biomass composition of $ CH_{1.8}O_{0.5}N_{0.2} $ and ammonia, $ NH_3 $ as carbon source, a prediction of the stoichiometry for the biomass formation reaction can be made by balancing out all elements in each of the reactions:</p>

<p>$ -r_{CO2} CO_2 - r_{H} H_2 - r_{Am} NH_3 + CH_{1.8}O_{0.5}N_{0.2} + r_{W} H_2O = 0$ </p>
<p>$ -r_{CO} CO - r_{W} H_2O - r_{Am} NH_3 + CH_{1.8}O_{0.5}N_{0.2} + r_{CO2} CO_2 = 0$</p>

Which results in:

<p>$ -CO_2 - 2.1 H_2 - 0.2 NH_3 + CH_{1.8}O_{0.5}N_{0.2} + 1.5 H_2O = 0$ </p>
<p>$ -2.1 CO - 0.6 H_2O - 0.2 NH_3 + CH_{1.8}O_{0.5}N_{0.2} + 1.1 CO_2 = 0$</p>

## Combine catabolic and anabolic reactions

Carboxydotrophic

In [6]:
r_carb_CO = -2.1+4*a_CO/4
r_carb_W = -0.6+2*a_CO/4
r_carb_Am = -0.2
r_carb_X = 1
r_carb_actt = -a_CO/4
r_carb_CO2 = 1.1-2*a_CO/4

print(f"Overall Carboxydotrophic stoichiometry at {T1}K: \n {round(r_carb_CO,2)} CO {round(r_carb_W,2)} H20 {round(r_carb_Am,2)} NH3 + {round(r_carb_X,2)} biomass + {round(r_carb_actt,2)} CH3COOH + {round(r_carb_CO2,2)} CO2 = 0 \n")

print("Yields on substrate:","\n",
      "r_carb_CO =", round(r_carb_CO,2), "\n",
      "r_carb_W =", round(r_carb_W,2), "\n",
      "r_carb_Am =", round(r_carb_Am,2), "\n",
      "r_carb_X =", round(r_carb_X,2), "\n",
      "r_carb_actt =", round(r_carb_actt,2), "\n",
      "r_carb_CO2 =", round(r_carb_CO2,2), "\n",
      "Y_carb_XS =", round(r_carb_X/r_carb_CO,3), "\n",
      "Y_carb_PS =", round(r_carb_actt/r_carb_CO,3), "\n",
      "Y_carb_PX =", round(r_carb_actt/r_carb_X,3))

Overall Carboxydotrophic stoichiometry at 303.15K: 
 -19.38 CO -9.24 H20 -0.2 NH3 + 1 biomass + 4.32 CH3COOH + 9.74 CO2 = 0 

Yields on substrate: 
 r_carb_CO = -19.38 
 r_carb_W = -9.24 
 r_carb_Am = -0.2 
 r_carb_X = 1 
 r_carb_actt = 4.32 
 r_carb_CO2 = 9.74 
 Y_carb_XS = -0.052 
 Y_carb_PS = -0.223 
 Y_carb_PX = 4.321


Homoacetogenic

In [7]:
r_hom_CO2 = -1 + 2*a_H2/4
r_hom_H2 = -2.1 + 4*a_H2/4
r_hom_Am = -0.2
r_hom_X = 1
r_hom_actt = -a_H2/4
r_hom_W = 1.5 - 2*a_H2/4

print(f"Overall Homoacetogenic stoichiometry at {T1}K: \n {round(r_hom_CO2,2)} CO2 {round(r_hom_H2,2)} H2 {round(r_hom_Am,2)} NH3 + {round(r_hom_X,2)} biomass + {round(r_hom_actt,2)} CH3COOH + {round(r_hom_W,2)} H2O = 0 \n")

print("Yields on substrate:","\n",
      "r_hom_CO2 =", round(r_hom_CO2,2), "\n",
      "r_hom_H2 =", round(r_hom_H2,2), "\n",
      "r_hom_Am =", round(r_hom_Am,2), "\n",
      "r_hom_X =", round(r_hom_X,2), "\n",
      "r_hom_actt =", round(r_hom_actt,2), "\n",
      "r_hom_W =", round(r_hom_W,2), "\n",
      "Y_homo_XS =", round(r_hom_X/r_hom_H2,3), "\n",
      "Y_homo_PS =", round(r_hom_actt/r_hom_H2,3), "\n",
      "Y_homo_PX =", round(r_hom_actt/r_hom_X,3))

Overall Homoacetogenic stoichiometry at 303.15K: 
 -14.26 CO2 -28.62 H2 -0.2 NH3 + 1 biomass + 6.63 CH3COOH + 14.76 H2O = 0 

Yields on substrate: 
 r_hom_CO2 = -14.26 
 r_hom_H2 = -28.62 
 r_hom_Am = -0.2 
 r_hom_X = 1 
 r_hom_actt = 6.63 
 r_hom_W = 14.76 
 Y_homo_XS = -0.035 
 Y_homo_PS = -0.232 
 Y_homo_PX = 6.631


## Calculate an estimate for maintenance requierment

REF: L. Tijhuis, M.C. M. van Loosdrecht, and J. J. Heijnen, A Thermodynamically Based Correlation for Maintenance Gibbs Energy Requirements in Aerobic and Anaerobic Chemotrophic Growth 

In [8]:
m_G = 3.3**(69000/R*(1/298-1/T1)) #[kJ/h/CmolX]
print("m_G =", round(m_G,2), "[kJ/h/CmolX]")

m_G = 1.76 [kJ/h/CmolX]


Convert to maintenance substrate requirement

In [9]:
m_carb = m_G/(DrGT_carb(T1)/4) #[molCO/h/CmolX] based on stoichiometry of electron donor
m_hom = m_G/(DrGT_hom(T1)/4) #[molCO/h/CmolX] based on stoichiometry of electron donor

print("Substrate required for maintenance:\n",
      "m_carb = ",round(m_carb,1), "[molCO/h/CmolX]\n",
      "m_hom = ",round(m_hom,1), "[molH2/h/CmolX]")

Substrate required for maintenance:
 m_carb =  -0.0 [molCO/h/CmolX]
 m_hom =  -0.0 [molH2/h/CmolX]


### Michaelis-Menten Parameters

In [10]:
Ks_h2 =  0.0002950 # [mM] https://doi.org/10.1016/j.jece.2022.107281
Ks_co2 =  0.0002950 # [mM] taken from H2
Ks_co = 0.0002950 # [mM] taken from H2

Ki_actt = 800 #mM Inhibition constant for acetone

µ_max_hom = 0.4 #[/h] https://doi.org/10.1016/j.jece.2022.107281
µ_max_carb = µ_max_hom

# Construct Black Box models of Moorella

<p>Monod Equation, specific growth rate: </p>
<div style="text-align: center">  $ µ = µ_{max} * 
\displaystyle \frac{c_s}{K_s + c_s} * (1-c_p/K_{ip})$ [$h^{-1}$]
    <span style="float:right;">(eq. 27)</span>
</div>

<p>Pirt kinetics, specific consumption and production rate: </p>
<br>
<div style="text-align: center">  $ q_i = \displaystyle \frac{µ}{Y_{Xi}} + m_{NGAM,i} $ [$h^{-1}$]
    <span style="float:right;">(eq. 28)</span>
</div>
<br>
<div style="text-align: center">  $ m_{NGAM,i} = 
\displaystyle \frac{m_{NGAM}}{\Delta _r G^T/q_i}$ [$h^{-1}$]
    <span style="float:right;">(eq. 29)</span>
</div>



def µ_hom(C_h2, C_co2, C_actt,T):
    return µ_max_hom * min(C_h2/(Ks_h2+C_h2), C_co2/(Ks_co2+C_co2)) * (1-C_actt/Ki_actt)

def q_hom_CO2(C_h2, C_co2, C_actt,T):
    m_NGAM_Hom_CO2  = S.HomCata["CO2"]/S.HomCata["H2"] * m_NGAM(T)/(DrGT_hom(T)/S.HomCata["H2"])
    return µ_hom(C_h2, C_co2, C_actt)/(r_hom["X"]/r_hom["CO2"]) + m_NGAM_Hom_CO2

def q_hom_H2(C_h2, C_co2, C_actt,T):
    m_NGAM_Hom_H2   = S.HomCata["H2"]/S.HomCata["H2"] * m_NGAM(T)/(DrGT_hom(T)/S.HomCata["H2"])
    return µ_hom(C_h2, C_co2, C_actt)/(r_hom["X"]/r_hom["H2"]) + m_NGAM_Hom_H2

def q_hom_actt(C_h2, C_co2, C_actt,T):
    m_NGAM_Hom_Actt = S.HomCata["P"]/S.HomCata["H2"] * m_NGAM(T)/(DrGT_hom(T)/S.HomCata["H2"])
    return µ_hom(C_h2, C_co2, C_actt)/(r_hom["X"]/r_hom["P"]) + m_NGAM_Hom_Actt

def µ_carb(C_co, C_actt,T):
    return µ_max_carb * C_co/(Ks_co+C_co) * (1-C_actt/Ki_actt)

def q_carb_CO(C_co, C_actt,T):
    m_NGAM_Carb_CO   = S.CarbCata["CO"]/S.CarbCata["CO"] * m_NGAM(T)/(DrGT_carb(T)/S.CarbCata["CO"])
    return µ_carb(C_co, C_actt)/(r_carb["X"]/r_carb["CO"]) + m_NGAM_Carb_CO

def q_carb_actt(C_co, C_actt,T):
    m_NGAM_Carb_Actt = S.CarbCata["P"]/S.CarbCata["CO"] * m_NGAM(T)/(DrGT_carb(T)/S.CarbCata["CO"])
    return µ_carb(C_co, C_actt)/(r_carb["X"]/r_carb["P"]) + m_NGAM_Carb_Actt

def q_carb_CO2(C_co, C_actt,T):
    m_NGAM_Carb_CO2  = S.CarbCata["CO2"]/S.CarbCata["CO"] * m_NGAM(T)/(DrGT_carb(T)/S.CarbCata["CO"])
    return µ_carb(C_co, C_actt)/(r_carb["X"]/r_carb["CO2"]) + m_NGAM_Carb_CO2

Get better parameters for gas uptake kinetics from BRENDA


CODH for CO: https://www.brenda-enzymes.org/enzyme.php?ecno=1.2.7.4 

formate dehydrogenase for CO2: https://www.brenda-enzymes.org/enzyme.php?ecno=1.17.1.9

ferredoxin hydrogenase for H2: https://www.brenda-enzymes.org/enzyme.php?ecno=1.12.7.2

Are they temperature dependent?

In [11]:
def µ_hom(C_h2, C_co2, C_actt, T):
    #µ_hom = 
    return µ_max_hom * min(C_h2/(Ks_h2+C_h2), C_co2/(Ks_co2+C_co2)) * (1-C_actt/Ki_actt)

def q_hom_CO2(C_h2, C_co2, C_actt, T):
    #q_hom_CO2 = µ_hom(C_h2, C_co2, C_actt)/(r_hom_X/r_hom_CO2) + (-2/-4) * m_hom
    return µ_hom(C_h2, C_co2, C_actt)/(r_hom_X/r_hom_CO2) + (-2/-4) * m_hom

def q_hom_H2(C_h2, C_co2, C_actt, T):
    #q_hom_H2 = µ_hom(C_h2, C_co2, C_actt)/(r_hom_X/r_hom_H2) + m_hom
    return µ_hom(C_h2, C_co2, C_actt)/(r_hom_X/r_hom_H2) + m_hom

def q_hom_actt(C_h2, C_co2, C_actt, T):
    #q_hom_actt = µ_hom(C_h2, C_co2, C_actt)/(r_hom_X/r_hom_actt) + (1/-4) * m_hom
    return µ_hom(C_h2, C_co2, C_actt)/(r_hom_X/r_hom_actt) + (1/-4) * m_hom

In [12]:
def µ_carb(C_co, C_actt, T):
    #µ_moth_carb = µ_max_carb * C_co/(Ks_co+C_co) * (1-C_actt/Ki_actt)
    return µ_max_carb * C_co/(Ks_co+C_co) * (1-C_actt/Ki_actt)

def q_carb_CO(C_co, C_actt, T):
    #q_carb_CO = µ_carb(C_co, C_actt)/(r_carb_X/r_carb_CO) + m_carb
    return µ_carb(C_co, C_actt)/(r_carb_X/r_carb_CO) + m_carb

def q_carb_actt(C_co, C_actt, T):
    #q_carb_actt = µ_carb(C_co, C_actt)/(r_carb_X/r_carb_actt) + (1/-4) * m_carb
    return µ_carb(C_co, C_actt)/(r_carb_X/r_carb_actt) + (1/-4) * m_carb

def q_carb_CO2(C_co, C_actt, T):
    #q_carb_CO2 = µ_carb(C_co, C_actt)/(r_carb_X/r_carb_CO2) + (2/-4) * m_carb
    return µ_carb(C_co, C_actt)/(r_carb_X/r_carb_CO2) + (2/-4) * m_carb

## Bubble column setup:

Process Parameters

In [13]:
F_G = 1*10**4 #gas inflow rate (from Steph: (8*10**5 * 8.314 * T)/p_m) [m^3/h]
r = 3 # radius of the reactor [m]
h = 30 # height of the reactor [m]

#deduceable constants
A = math.pi * r**2 #reactor cross-sectional area [m^2]
V_L = A * h * 2/3 #reactor volume, assuming it is 2/3 filled [m^3]
v_gas = F_G/A/3600 #average gas velocity [m/s]
Gas_holdup = 0.6*(v_gas)**0.7 #from Stephanie's paper
V_G = V_L*Gas_holdup

p_t = 101325 # pressure at the top of the reactor [Pa]
p_b = p_t + h * 1000 * 9.81 # pressure at the bottom of the reactor [Pa]
p_m = (p_b-p_t)/math.log(p_b/p_t) # logarithmic mean pressure [Pa]

Define gas transfer rates as functions

In [14]:
pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")

,DR,HR,k
Compound,,,
CO,0.000020,0.99,1300
CO2,0.000019,35.00,2400
H2,0.000045,0.78,500
O2,0.000021,1.30,1500
N2,0.000019,0.60,1300


In [15]:
DR = pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")["DR"]
HR = pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")["HR"]
kh = pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")["k"]
theta = 1.022 # temperature correction factor

#Kla Temperature correction:
def KLa_CO2(t):
    return 0.32 * (DR["CO2"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(t-293.15) * 3600 #[1/s]

def KLa_CO(t):
    return 0.32 * (DR["CO"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(t-293.15) * 3600 #[1/s]

def KLa_H2(t):
    return 0.32 * (DR["H2"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(t-293.15) * 3600 #[1/s]

def KLa_N2(t):
    return 0.32 * (DR["N2"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(t-293.15) * 3600 #[1/s]

#Saturation concentration
def CO2_sat(t):
    return HR["CO2"] * math.e**(kh["CO2"]*(1/t-1/298.15)) * 10**(-5) * p_m

def CO_sat(t):
    return HR["CO"] * math.e**(kh["CO"]*(1/t-1/298.15)) * 10**(-5) * p_m

def H2_sat(t):
    return HR["H2"] * math.e**(kh["H2"]*(1/t-1/298.15)) * 10**(-5) * p_m

def N2_sat(t):
    return HR["N2"] * math.e**(kh["N2"]*(1/t-1/298.15)) * 10**(-5) * p_m

#Combine as transfer rate:
def R_CO2(c_co2,cG_co2,cG_Gas,t):
    return KLa_CO2(t) * (CO2_sat(t) - c_co2)

def R_CO(c_co,cG_co,cG_Gas,t):
    return KLa_CO(t) * (CO_sat(t) - c_co)

def R_H2(c_h2,cG_h2,cG_Gas,t):
    return KLa_H2(t) * (H2_sat(t) - c_h2)

def R_N2(c_n2,cG_n2,cG_Gas,t):
    return KLa_N2(t) * (N2_sat(t) - c_n2)

# Load Henry's Law Constants:
DR = pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")["DR"]
HR = pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")["HR"]
kh = pd.read_csv("../Data/HenrysLawData.csv", index_col = "Compound")["k"]
theta = 1.022 # temperature correction factor

# Henry's law constant: 
HT_CO2 = HR["CO2"] * math.e**(kh["CO2"]*(1/T-1/298.15)) * 10**(-5) #[mol/m^3*Pa]
HT_CO = HR["CO"] * math.e**(kh["CO"]*(1/T-1/298.15)) * 10**(-5) #[mol/m^3*Pa]
HT_H2 = HR["H2"] * math.e**(kh["H2"]*(1/T-1/298.15)) * 10**(-5) #[mol/m^3*Pa]
HT_N2 = HR["N2"] * math.e**(kh["N2"]*(1/T-1/298.15)) * 10**(-5) #[mol/m^3*Pa]

KLa_CO2 = 0.32 * (DR["CO2"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(T-293.15) * 3600 #[1/s]
KLa_CO = 0.32 * (DR["CO"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(T-293.15) * 3600 #[1/s]
KLa_H2 = 0.32 * (DR["H2"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(T-293.15) * 3600 #[1/s]
KLa_N2 = 0.32 * (DR["N2"]/DR["O2"])**0.5 * v_gas**0.7 * theta**(T-293.15) * 3600 #[1/s]

# Transfer rate
def R_CO2(c_co2,cG_co2,cG_Gas):
    return KLa_CO2 * (HT_CO2 * p_m - c_co2)

def R_CO(c_co,cG_co,cG_Gas):
    return KLa_CO * (HT_CO * p_m - c_co)

def R_H2(c_h2,cG_h2,cG_Gas):
    return KLa_H2 * (HT_H2 * p_m - c_h2)

def R_N2(c_n2,cG_n2,cG_Gas):
    return KLa_N2 * (HT_N2 * p_m - c_n2)

<div><b>Box 1:</b> Mass Balances; Accumulation = In - Out + Production - Consumption</div>
<br>
<div>Biomass: &emsp; &emsp; &emsp; &ensp; $ \displaystyle \frac{dc_X}{dt} = µ * c_X $
<div>Acetate: &emsp; &emsp; &emsp; &emsp; $ \displaystyle \frac{dc_{Actt}}{dt} = q_{actt} * c_X$
<div>CO<sub>2</sub>, dissolved:&emsp;&nbsp; $ \displaystyle \frac{dc_{CO_2}}{dt} = q_{CO_2} * c_X + R_{CO_2} $
<div>CO, dissolved: &emsp; &nbsp; $ \displaystyle \frac{dc_{CO}}{dt} = q_{CO} * c_X + R_{CO}$
<div>H<sub>2</sub>, dissolved:&emsp;&emsp;&ensp;&nbsp;$ \displaystyle \frac{dc_{H_2}}{dt} = q_{H_2} * c_X + R_{H_2} $
<div>CO<sub>2</sub>, gas:&emsp;&emsp;&emsp;&emsp;&emsp;$ \displaystyle \frac{dc_{G,CO_{2}}}{dt}=\frac{F_G}{V_G}*(c_{G,CO_2,in}-c_{G,CO_2})-R_{CO_2}*\frac{V_L}{V_G}$
<div>CO, gas:&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;$ \displaystyle \frac{dc_{G,CO}}{dt}=\frac{F_G}{V_G}*(c_{G,CO,in}-c_{G,CO})-R_{CO}*\frac{V_L}{V_G} $
<div>H<sub>2</sub>, gas:&emsp;&emsp;&emsp;&emsp;&emsp;&ensp;&nbsp;$ \displaystyle \frac{dc_{G,H_{2}}}{dt}=\frac{F_G}{V_G}*(c_{G,H_2,in}-c_{G,H_2})-R_{H_2}*\frac{V_L}{V_G}$

In [16]:
def model(z,t):
    X_carb = max(0,z[0])
    X_hom = max(0,z[1])
    c_actt = max(0,z[2])
    c_co2 = max(0,z[3])
    c_co = max(0,z[4])
    c_h2 = max(0,z[5])
    c_n2 = max(0,z[6])
    cG_co2 = max(0,z[7])
    cG_co = max(0,z[8])
    cG_h2 = max(0,z[9])
    cG_n2 = max(0,z[10])
    
    cG_Gas = cG_co2+cG_co+cG_h2+cG_n2

    dXcdt = µ_carb(c_co, c_actt,T) * X_carb
    dXhdt = µ_hom(c_h2, c_co2, c_actt,T) * X_hom
    dacttdt = q_hom_actt(c_h2, c_co2, c_actt,T) * X_hom + q_carb_actt(c_co, c_actt,T) * X_carb
    dCO2dt = q_hom_CO2(c_h2, c_co2, c_actt,T) * X_hom + q_carb_CO2(c_co, c_actt,T) * X_carb + R_CO2(c_co2,cG_co2,cG_Gas,T)
    dCOdt = q_carb_CO(c_co, c_actt,T) * X_carb + R_CO(c_co,cG_co,cG_Gas,T)
    dH2dt = q_hom_H2(c_h2, c_co2, c_actt,T) * X_hom + R_H2(c_h2,cG_h2,cG_Gas,T)
    dN2dt = R_N2(c_n2,cG_n2,cG_Gas,T)
    dCO2Gdt = F_G/V_G * (cG_co2_in-cG_co2) - R_CO2(c_co2,cG_co2,cG_Gas,T) * V_L/V_G
    dCOGdt = F_G/V_G * (cG_co_in-cG_co) - R_CO(c_co,cG_co,cG_Gas,T) * V_L/V_G
    dH2Gdt = F_G/V_G * (cG_h2_in-cG_h2) - R_H2(c_h2,cG_h2,cG_Gas,T) * V_L/V_G
    dN2Gdt = F_G/V_G * (cG_n2_in-cG_n2) - R_N2(c_n2,cG_n2,cG_Gas,T) * V_L/V_G
    
    return [dXcdt,dXhdt,dacttdt,dCO2dt,dCOdt,dH2dt,dN2dt,dCO2Gdt,dCOGdt,dH2Gdt,dN2Gdt]

For the initial gas concentrations: 
 - in the liquid, one can assume that they have reached saturation before starting the experiment
 - in the gas, one can take the inlet concentrations

In [17]:
#initial conditions at T1
T = T1

cG_co2_in = y_co2 * p_m/(R*T)
cG_co_in = y_co * p_m/(R*T)
cG_h2_in = y_h2 * p_m/(R*T)
cG_n2_in = y_n2 * p_m/(R*T)

c_co2_0 = CO2_sat(T)
c_co_0 =  CO_sat(T)
c_h2_0 =  H2_sat(T)
c_n2_0 =  N2_sat(T)

cG_Gas_0 = cG_co2_in+cG_co_in+cG_h2_in+cG_n2_in

#z0=[c_n2_0,cG_n2_in]
z0=[0.1,0.1,0,c_co2_0,c_co_0,c_h2_0,c_n2_0,cG_co2_in,cG_co_in,cG_h2_in,cG_n2_in]

#step function
t_rt = 30
t = np.linspace(0,t_rt,1000)

#solve ODE
z_T1 = odeint(model,z0,t)

TypeError: μ_hom() missing 1 required positional argument: 'T'

In [ ]:
#initial conditions at T2
T = T2

cG_co2_in = y_co2 * p_m/(R*T)
cG_co_in = y_co * p_m/(R*T)
cG_h2_in = y_h2 * p_m/(R*T)
cG_n2_in = y_n2 * p_m/(R*T)

c_co2_0 = CO2_sat(T)
c_co_0 =  CO_sat(T)
c_h2_0 =  H2_sat(T)
c_n2_0 =  N2_sat(T)

cG_Gas_0 = cG_co2_in+cG_co_in+cG_h2_in+cG_n2_in

#z0=[c_n2_0,cG_n2_in]
z0=[0.1,0.1,0,c_co2_0,c_co_0,c_h2_0,c_n2_0,cG_co2_in,cG_co_in,cG_h2_in,cG_n2_in]

#step function
t_rt = 30
t = np.linspace(0,t_rt,1000)

#solve ODE
z_T2 = odeint(model,z0,t)

In [ ]:
sim = 1
#sim = 2

fig, ax1 = plt.subplots(figsize=(12,4))

plt.rcParams['axes.spines.right'] = True
plt.rcParams['axes.spines.top'] = False
plt.title("C", fontdict = {'fontsize': 20}, loc = "left")
#plt.title(f"Acetogenic Growth Simulation in Bubble Column at {Tc}°C")
color = 'tab:red'
ax1.set_xlabel(r'Time [$h$]', fontsize=15)
ax1.set_ylabel(r'Biomass [$gDW/l$]', color=color, fontsize=15)
ax1.plot(t, (z_T1[:,0]+z_T1[:,1])*24.6/1000, color=color) #Sum of carboxydotrophic and homoacetogenic, converted to gDW assuming C1 H1.8 O0.5 N0.2
ax1.plot(t, (z_T2[:,0]+z_T2[:,1])*24.6/1000, color=color)
ax1.axis([0, t_rt, 0, 2.8])
ax1.tick_params(axis='y', labelcolor=color)
ax1.tick_params(axis='both', which='major', labelsize=15)

ax2 = ax1.twinx() # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel(r'Acetate [$mM$]', color=color, fontsize=15)  # we already handled the x-label with ax1
ax2.plot(t, z_T1[:,2], color=color)
ax2.plot(t, z_T2[:,2], color=color)
ax2.axis([0, t_rt, 0, 1000])
ax2.tick_params(axis='y', labelcolor=color)
ax2.tick_params(axis='both', which='major', labelsize=15)
plt.rcParams['axes.spines.top'] = False

fig.tight_layout() # otherwise the right y-label is slightly clipped
plt.show()

fig = plt.figure(figsize=(12,5.4))

plt.rcParams['axes.spines.right'] = False
#plt.rcParams['axes.spines.top'] = False
plt.subplot(2,2,1)
plt.title("D", fontdict = {'fontsize': 20}, loc = "left")
plt.plot(t,z_T1[:,3])
plt.plot(t,z_T1[:,4])
plt.plot(t,z_T1[:,5])
plt.plot(t,z_T1[:,6])
plt.xlabel(r'Time [$h$]', fontsize=15)
plt.ylabel(r'Disolved gas [$mM$]', fontsize=15)
plt.axis([0, t_rt, 0, 2.9])
plt.tick_params(axis='both', which='major', labelsize=15)
#plt.legend(['CO2','CO',"H2","N2"],bbox_to_anchor=(0.82, 0.6))

plt.subplot(2,2,3)
plt.title("E", fontdict = {'fontsize': 20}, loc = "left")
plt.plot(t,z_T1[:,7])
plt.plot(t,z_T1[:,8])
plt.plot(t,z_T1[:,9])
plt.plot(t,z_T1[:,10])
plt.xlabel(r'Time [$h$]', fontsize=15)
plt.ylabel(r'Undisolved gas [$mM$]', fontsize=15)
plt.axis([0, t_rt, 0, 40])
plt.tick_params(axis='both', which='major', labelsize=15)

fig.tight_layout()

plt.subplot(2,2,2)
plt.title("G", fontdict = {'fontsize': 20}, loc = "left")
plt.plot(t,z_T2[:,3])
plt.plot(t,z_T2[:,4])
plt.plot(t,z_T2[:,5])
plt.plot(t,z_T2[:,6])
plt.xlabel(r'Time [$h$]', fontsize=15)
plt.ylabel(r'Disolved gas [$mM$]', fontsize=15)
plt.axis([0, t_rt, 0, 2.9])
plt.tick_params(axis='both', which='major', labelsize=15)
plt.legend(['CO2','CO',"H2","N2"],bbox_to_anchor=(0.82, 0.6))

plt.subplot(2,2,4)
plt.title("H", fontdict = {'fontsize': 20}, loc = "left")
plt.plot(t,z_T2[:,7])
plt.plot(t,z_T2[:,8])
plt.plot(t,z_T2[:,9])
plt.plot(t,z_T2[:,10])
plt.xlabel(r'Time [$h$]', fontsize=15)
plt.ylabel(r'Undisolved gas [$mM$]', fontsize=15)
plt.axis([0, t_rt, 0, 40])
plt.tick_params(axis='both', which='major', labelsize=15)
fig.tight_layout()